# Import statements & utilities

In [ ]:
!pip install -q transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


In [ ]:
pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
from urllib import request
import pandas as pd
from collections import Counter
import random
import numpy as np
import pandas as pd
import copy
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

# Load preprocessed data

In [ ]:
# if data already saved-
trdf1 = pd.read_csv("trdf1.csv", index_col=0)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [ ]:
tedf1 = pd.read_csv("tedf1.csv", index_col=0)
tedf1

,par_id,community,text,label
0,9830,hopeless,Since Mr. Cruz 's election to the Senate in 20...,0
1,9165,refugee,These are some of the dilemmas that befell Min...,0
2,9265,immigrant,"Lucy Lawless ( Salem , Spartacus ) as Ruby , a...",0
3,8457,refugee,The situation has some parallels to Myanmar 's...,0
4,10122,poor-families,"Five years down the lane , the schools whose l...",0
...,...,...,...,...
2089,9170,migrant,"This May 5 , 2018 , photo , released by the Ro...",0
2090,9264,vulnerable,"""Asim Qureshi , of Cageprisoners , says that h...",0
2091,9396,immigrant,Mr Ariel told ministry of housing staff and of...,0
2092,2953,poor-families,""""""" The worst thing was dealt to us , """" he sa...",1


# OpenAI API setup

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key="",  # ENTER API KEY HERE
)

In [ ]:
def translate_text(text):
    completion = client.chat.completions.create(model="gpt-3.5-turbo",
        messages=[
        {
            "role": "user",
            "content": f"Translate the following text into spanish: {text}\n",
        },
    ],
    n=1,
    temperature=0.7)
    return completion.choices[0].message.content

In [ ]:
def backtranslate(text):
      completion = client.chat.completions.create(model="gpt-3.5-turbo",
          messages=[
          {
              "role": "user",
              "content": f"Translate the following text into english: {text}\n",
          },
      ],
      n=1,
      temperature=0.7)
      return completion.choices[0].message.content

In [ ]:
text = "Hello world!"

completion = client.chat.completions.create(model="gpt-3.5-turbo",
        messages=[
        {
            "role": "user",
            "content": f"Translate the following text into spanish: {text}\n",
        },
    ],
    n=1,
    temperature=0.7)
print(completion.choices[0].message.content)

¡Hola mundo!


In [ ]:
trial = trdf1.head()

In [ ]:
trial["text_spanish"] = trial["text"].apply(translate_text)

<ipython-input-28-fe28b37fab75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial["text_spanish"] = trial["text"].apply(translate_text)


In [ ]:
trial

,par_id,community,text,label,text_spanish
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1,"El programa vio a aproximadamente 150,000 niño..."
1,4136,homeless,Durban 's homeless communities reconciliation ...,1,Almuerzo de reconciliación de las comunidades ...
2,10352,poor-families,The next immediate problem that cropped up was...,1,El siguiente problema inmediato que surgió fue...
3,8279,vulnerable,Far more important than the implications for t...,1,Mucho más importante que las implicaciones par...
4,1164,poor-families,To strengthen child-sensitive social protectio...,1,Para fortalecer los sistemas de protección soc...


In [ ]:
trial["text_spanish"][0]

'El programa vio a aproximadamente 150,000 niños de familias pobres ser enviados a diferentes partes del Imperio Británico entre 1920 y 1974, por órdenes religiosas y organizaciones benéficas que aseguraban que llevarían una vida mejor.'

In [ ]:
df_to_augment = (trdf1[trdf1.label==1]).copy()
df_to_augment["text_spanish"] = ""

In [ ]:
df_to_augment

,par_id,community,text,label,text_spanish
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1,
1,4136,homeless,Durban 's homeless communities reconciliation ...,1,
2,10352,poor-families,The next immediate problem that cropped up was...,1,
3,8279,vulnerable,Far more important than the implications for t...,1,
4,1164,poor-families,To strengthen child-sensitive social protectio...,1,
...,...,...,...,...,...
789,873,poor-families,Citing the fact that these kids who died at Go...,1,
790,10070,disabled,Fern ? ndez was a well-known philanthropist wh...,1,
791,6484,homeless,"Touched much by their plight , Commanding Offi...",1,
792,6249,women,She reiterated her ministry 's commitment to p...,1,


In [ ]:
df_to_augment["text_spanish"] = df_to_augment["text"].apply(translate_text)

In [ ]:
df_to_augment

,par_id,community,text,label,text_spanish
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1,"El plan vio aproximadamente 150,000 niños de f..."
1,4136,homeless,Durban 's homeless communities reconciliation ...,1,Almuerzo de reconciliación de las comunidades ...
2,10352,poor-families,The next immediate problem that cropped up was...,1,El siguiente problema inmediato que surgió fue...
3,8279,vulnerable,Far more important than the implications for t...,1,Mucho más importante que las implicaciones par...
4,1164,poor-families,To strengthen child-sensitive social protectio...,1,Para fortalecer los sistemas de protección soc...
...,...,...,...,...,...
789,873,poor-families,Citing the fact that these kids who died at Go...,1,Citando el hecho de que estos niños que murier...
790,10070,disabled,Fern ? ndez was a well-known philanthropist wh...,1,Fernández fue un conocido filántropo que nunca...
791,6484,homeless,"Touched much by their plight , Commanding Offi...",1,"Conmovidos por su difícil situación, el Oficia..."
792,6249,women,She reiterated her ministry 's commitment to p...,1,Ella reiteró el compromiso de su ministerio de...


In [ ]:
df_to_augment.to_csv("df_to_augment.csv")

In [ ]:
columns = df_to_augment.columns
augmented_data = pd.DataFrame(columns=["community", "text", "label"])

# Display the new DataFrame
augmented_data

,community,text,label


In [ ]:
for index, row in df_to_augment.iterrows():
    line = {}
    for column_name, item in row.iteritems():
        if column_name == "text_spanish":
            text = backtranslate(item)
            line["text"] = text
        elif column_name in ["par_id", "text"]:
            continue
        else:
            line[column_name] = item
    augmented_data = augmented_data.append(line, ignore_index=True)


<ipython-input-18-539ada54a43c>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, item in row.iteritems():
<ipython-input-18-539ada54a43c>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(line, ignore_index=True)
<ipython-input-18-539ada54a43c>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, item in row.iteritems():
<ipython-input-18-539ada54a43c>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(line, ignore_index=True)
<ipython-input-18-539ada54a43c>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, item in row.iterit

In [ ]:
augmented_data

,community,text,label
0,poor-families,"The plan saw approximately 150,000 children fr...",1
1,homeless,Lunch of reconciliation for the homeless commu...,1
2,poor-families,The next immediate problem that arose was how ...,1
3,vulnerable,Much more important than the implications for ...,1
4,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...
789,poor-families,Quoting the fact that these children who died ...,1
790,disabled,Fernández was a well-known philanthropist who ...,1
791,homeless,"Moved by their difficult situation, the Comman...",1
792,women,She reiterated her ministry's commitment to im...,1


In [ ]:
augmented_data.to_csv("pcl_augmented_data.csv")